In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# 1. 데이터 셋 가져오기

In [22]:
import pandas as pd
df = pd.read_csv('../01_python/data/ch13_df(결측치중위값으로대체).csv')
df.head()

,지역명,연도월,평당분양가격,연도,월
0,서울,2013년12월,18189.0,2013,12
1,부산,2013년12월,8111.0,2013,12
2,대구,2013년12월,8080.0,2013,12
3,인천,2013년12월,10204.0,2013,12
4,광주,2013년12월,6098.0,2013,12


- 지역명2 : 지역명필드를 라벨인코딩하여 추가(LabelIncoder)
- 독립변수 : 지역명2, 연도, 월 (x)
- 타겟변수 : 평당분양가격      (y)
- 독립변수와 타겟변수의 스케일 조정
    * 정규화(MinMaxScaler) 작업후 : 지역명2m, 연도m, 월m
    * 표준화(StandardScaler) 작업후 : 지역명2s, 연도s, 월s
  => 지역명, 연도, 월, 지역명2, 지역명2m, 연도m, 월m, 지역명2s, 연도s, 월s

# 2. 지역명의 라벨 인코딩
- 지역명2

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# le.fit(df['지역명'])
# le.transform(df['지역명'])
df['지역명2'] = le.fit_transform(df['지역명'])   # 문자데이터를 가나다순으로 정렬후 숫자로 변환
df.head()

,지역명,연도월,평당분양가격,연도,월,지역명2
0,서울,2013년12월,18189.0,2013,12,8
1,부산,2013년12월,8111.0,2013,12,7
2,대구,2013년12월,8080.0,2013,12,5
3,인천,2013년12월,10204.0,2013,12,11
4,광주,2013년12월,6098.0,2013,12,4


# 3. MinMaxScale 조정

In [24]:
X_data = df[['지역명2','연도','월']].values  # numpy배열로 추출한 독립변수
y_data = df[['평당분양가격']].to_numpy()     # values나 to_numpy()를 이용해서 데이터 프레임을 numpy배열로 변환  
X_data.shape, y_data.shape

((2176, 3), (2176, 1))

In [25]:
y_data   # 2차원 넘파이배열

array([[18189. ],
       [ 8111. ],
       [ 8080. ],
       ...,
       [13827. ],
       [13252.8],
       [25419.9]])

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
df[['지역명2m','연도m','월m']] = scaler_x.fit_transform(X_data)
df[['평당분양가격m']] = scaler_y.fit_transform(y_data)
df.head()

,지역명,연도월,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m
0,서울,2013년12월,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198
1,부산,2013년12월,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274
2,대구,2013년12월,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466
3,인천,2013년12월,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878
4,광주,2013년12월,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757


# 4. StandardScale 조정

In [27]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
scaler_y = StandardScaler()
df[['지역명2s','연도s','월s']] = scaler_x.fit_transform(X_data)
df[['평당분양가격s']] = scaler_y.fit_transform(y_data)
df.head()

,지역명,연도월,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,2013년12월,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.172081
1,부산,2013년12월,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.722816
2,대구,2013년12월,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.728645
3,인천,2013년12월,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.329284
4,광주,2013년12월,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.101307


# 5. 지역명을 원핫인코딩
- 강원 0 => 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
- 서울 8 => 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0

In [31]:
loc = df.loc[:16,['지역명', '지역명2']].sort_values(by='지역명2')['지역명'].tolist() # 시리즈를 리스트로
print(loc)  # 원한인코딩 후 열이름

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [38]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
# to_categorical()의 결과는 numpy배열, 최대정수가 16이면 0~16 카테고리로 원핫인코딩, 매개변수에 정수만
# pd.get_dummies()의 결과는 데이터 프레임 , 정수가 곧 카테고리가 되서 원핫인코딩, 매개변수에 문자(라벨인코딩안 된값으로)
temp1 = to_categorical(df['지역명2'])  # 라벨인코딩된 열이름으로  
temp2 = pd.get_dummies(df['지역명']).values
np.all(temp1 == temp2)

True

In [44]:
# 원핫인코딩 방법1
df[loc] = to_categorical(df['지역명2'])# astype(np.int16)
pd.options.display.max_columns = 30
df[loc].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   강원      2176 non-null   float32
 1   경기      2176 non-null   float32
 2   경남      2176 non-null   float32
 3   경북      2176 non-null   float32
 4   광주      2176 non-null   float32
 5   대구      2176 non-null   float32
 6   대전      2176 non-null   float32
 7   부산      2176 non-null   float32
 8   서울      2176 non-null   float32
 9   세종      2176 non-null   float32
 10  울산      2176 non-null   float32
 11  인천      2176 non-null   float32
 12  전남      2176 non-null   float32
 13  전북      2176 non-null   float32
 14  제주      2176 non-null   float32
 15  충남      2176 non-null   float32
 16  충북      2176 non-null   float32
dtypes: float32(17)
memory usage: 144.6 KB


In [48]:
df[loc] = df[loc].astype(np.int16)
df.head()

,지역명,연도월,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,...,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013년12월,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.172081,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,2013년12월,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.722816,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,2013년12월,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.728645,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,2013년12월,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.329284,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,2013년12월,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.101307,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
# 원핫인코딩 방법2
df[loc] = pd.get_dummies(df['지역명2'])
df.head(1)

,지역명,연도월,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,...,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013년12월,18189.0,2013,12,8,0.5,0.0,1.0,0.328198,0.0,-1.875367,1.62196,1.172081,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [53]:
# 원핫인코딩 데이터 지우기
df = df.drop(loc, axis=1)
df.head(1)

,지역명,연도월,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,2013년12월,18189.0,2013,12,8,0.5,0.0,1.0,0.328198,0.0,-1.875367,1.62196,1.172081


In [55]:
# 원핫인코딩 방법3
df = pd.concat([df, pd.get_dummies(df['지역명'])], axis=1)
df.sample(2)

,지역명,연도월,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s,강원,...,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
1821,경기,NaN,17169.9,2022,12,1,0.0625,0.818182,1.0,0.301611,-1.428869,1.020642,1.62196,0.980466,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,광주,2013년12월,6098.0,2013,12,4,0.2500,0.000000,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.101307,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
